In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    100, shuffle=True)


log_interval  = 100



In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 10)

    def forward(self, x):
        print (x.size())
        x = x.view(-1,784)
        print(x.size())
        x = F.relu(self.fc1(x))
        print (x.size())
        x= F.softmax(x)
        print(x.size())
        return x
    
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [14]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.0001)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        print(data.size())
        optimizer.zero_grad()
        output = model(data)
        print(output[0:10,:])
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

        return  

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

  
train(1)

# for epoch in range(1, 100):
#     train(epoch)
#     test(epoch)
#     torch.save(model,'./mnist.model')

torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 10])
torch.Size([100, 10])
Variable containing:
 0.0741  0.1179  0.0741  0.0819  0.1186  0.0741  0.0913  0.0741  0.0958  0.1981
 0.0771  0.0956  0.0663  0.0663  0.0694  0.0841  0.2188  0.0663  0.1053  0.1508
 0.0855  0.1495  0.0855  0.0855  0.1175  0.0928  0.1273  0.0855  0.0855  0.0855
 0.0681  0.2263  0.0681  0.0681  0.0696  0.1161  0.1397  0.0681  0.1079  0.0681
 0.1665  0.0695  0.0933  0.0695  0.1598  0.1029  0.1047  0.0695  0.0695  0.0946
 0.1548  0.1283  0.0629  0.0629  0.1173  0.0629  0.0914  0.0629  0.0736  0.1829
 0.1028  0.1250  0.0735  0.1031  0.1164  0.0550  0.0586  0.0550  0.2372  0.0733
 0.0807  0.1269  0.1125  0.1310  0.1160  0.0823  0.1075  0.0807  0.0807  0.0817
 0.0758  0.1450  0.0758  0.0758  0.0948  0.1307  0.1071  0.0887  0.1306  0.0758
 0.0639  0.1733  0.0639  0.0639  0.0853  0.0639  0.1717  0.0639  0.1740  0.0763
[torch.FloatTensor of size 10x10]

Train Epoch: 1 [0/6